In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2
import matplotlib.pyplot as plt

c:\Users\ASUS\.conda\envs\dnn\Lib\site-packages\tensorflow_hub\__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


In [2]:
print(tf.__version__)

2.20.0


In [ ]:
I3D_PATH = r"C:\tfhub_models\i3d_kinetics_400"

base_model = hub.KerasLayer(
    I3D_PATH,
    trainable=False
)

In [7]:
video = np.random.rand(1, 64, 224, 224, 3).astype(np.float32)
out = base_model(video)

print(out.shape)


(1, 400)


In [ ]:
import cv2
import numpy as np



IMG_SIZE = 224
NUM_FRAMES = 64

def load_video_i3d(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
        frame = frame[..., ::-1]      # BGR → RGB
        frame = frame / 255.0         # normalize
        frames.append(frame)

    cap.release()
    frames = np.array(frames)

    # Pad or trim to NUM_FRAMES
    if len(frames) < NUM_FRAMES:
        pad_len = NUM_FRAMES - len(frames)
        frames = np.pad(frames, ((0, pad_len), (0,0), (0,0), (0,0)))
    else:
        frames = frames[:NUM_FRAMES]

    return frames.astype(np.float32)


In [10]:
video_path = "../test_videos/24.mp4"

video = load_video_i3d(video_path)
print(video.shape)


(64, 224, 224, 3)


In [11]:
video_batch = np.expand_dims(video, axis=0)  # (1, 64, 224, 224, 3)

features = base_model(video_batch)
print(features.shape)


(1, 400)


In [15]:
import tensorflow as tf

NUM_FRAMES = 64

inputs = tf.keras.Input(shape=(NUM_FRAMES, 224, 224, 3))

x = tf.keras.layers.Lambda(
    lambda t: base_model({"rgb_input": t})
)(inputs)

x = tf.keras.layers.Dense(256, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(2, activation="softmax")(x)

inference_model = tf.keras.Model(inputs, outputs)


In [16]:
inference_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [17]:
video = load_video_i3d("../test_videos/24.mp4")
video_batch = video[np.newaxis, ...]

prediction = inference_model.predict(video_batch)
print(prediction)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 871ms/step
[[9.9997747e-01 2.2507873e-05]]


In [18]:
labels = ["REAL", "FAKE"]

cls = prediction.argmax(axis=1)[0]
conf = prediction[0][cls]

print("Prediction:", labels[cls])
print("Confidence:", round(float(conf), 3))


Prediction: REAL
Confidence: 1.0


In [1]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("GPU:", torch.cuda.get_device_name(0))


CUDA available: True
CUDA version: 12.1
GPU: NVIDIA GeForce RTX 3050 6GB Laptop GPU


In [2]:
import torch
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("CUDA available:", torch.cuda.is_available())
print("Using device:", device)
print("GPU:", torch.cuda.get_device_name(0))

# Create big tensors to force GPU usage
a = torch.randn(5000, 5000, device=device)
b = torch.randn(5000, 5000, device=device)

print("Running matrix multiplication...")
for i in range(10):
    c = torch.matmul(a, b)
    time.sleep(0.5)

print("Done")


CUDA available: True
Using device: cuda
GPU: NVIDIA GeForce RTX 3050 6GB Laptop GPU
Running matrix multiplication...
Done
